In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import tree
import pydotplus
from IPython.display import Image
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
plt.interactive(False)
%matplotlib inline
import random
random.seed(42)

# Adults income

In [2]:
#https://www.kaggle.com/wenruliu/adult-income-dataset
df = pd.read_csv('adult.csv')
df.sample(5).T


,43795,47120,24634,2621,26354
age,34,42,26,36,62
workclass,Private,Private,Private,Private,Private
fnlwgt,346034,118947,120238,30269,57970
education,Bachelors,Bachelors,Bachelors,Some-college,7th-8th
educational-num,13,13,13,10,4
marital-status,Never-married,Married-civ-spouse,Never-married,Married-civ-spouse,Divorced
occupation,Other-service,Exec-managerial,Craft-repair,Other-service,Handlers-cleaners
relationship,Not-in-family,Husband,Not-in-family,Wife,Not-in-family
race,White,White,White,White,White
gender,Male,Male,Male,Female,Male


In [3]:
df['income'].value_counts(normalize=True)

<=50K    0.760718
>50K     0.239282
Name: income, dtype: float64

# Remove fair features

In [4]:
target = np.where(df['income'] == '<=50K', 0, 1)
df = df[['age', 'race', 'gender', 'native-country', 'marital-status', 'relationship']]

df['age'] = pd.cut(df['age'], 5)

In [5]:
df.describe(include='all').T

,count,unique,top,freq
age,48842,5,"(31.6, 46.2]",18277
race,48842,5,White,41762
gender,48842,2,Male,32650
native-country,48842,42,United-States,43832
marital-status,48842,7,Married-civ-spouse,22379
relationship,48842,6,Husband,19716


In [6]:
# normalized 
df = pd.get_dummies(df, columns=['age', 'race', 'gender', 'native-country', 'marital-status', 'relationship'] ,prefix='f')
df

,"f_(16.927, 31.6]","f_(31.6, 46.2]","f_(46.2, 60.8]","f_(60.8, 75.4]","f_(75.4, 90.0]",f_Amer-Indian-Eskimo,f_Asian-Pac-Islander,f_Black,f_Other,f_White,...,f_Married-spouse-absent,f_Never-married,f_Separated,f_Widowed,f_Husband,f_Not-in-family,f_Other-relative,f_Own-child,f_Unmarried,f_Wife
0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
48838,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
48839,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
48840,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0


# LR

In [7]:
%%time
lr_classifier = LogisticRegression(max_iter=1000, n_jobs=1) #class_weight='balanced',
scores = cross_val_score(lr_classifier, df, target, cv=10)
scores.mean()


Wall time: 8.3 s


0.7764627254033599

# DT

In [13]:
tree_classifier = tree.DecisionTreeClassifier(max_depth=8, min_samples_leaf=3, min_samples_split=3)
scores = cross_val_score(tree_classifier, df, target, cv=10)
scores.mean()


0.7727773558428626